In [4]:
import numpy as np
import pandas as pd

# Data Source
import yfinance as yf

# Data viz
import plotly.graph_objs as go

In [5]:
data = yf.download(tickers='UBER', period='5y')#, interval='1m')

[*********************100%***********************]  1 of 1 completed


In [8]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-05-10,42.000000,45.000000,41.060001,41.570000,41.570000,186322500
2019-05-13,38.790001,39.240002,36.080002,37.099998,37.099998,79442400
2019-05-14,38.310001,39.959999,36.849998,39.959999,39.959999,46661100
2019-05-15,39.369999,41.880001,38.950001,41.290001,41.290001,36086100
2019-05-16,41.480000,44.060001,41.250000,43.000000,43.000000,38115500
...,...,...,...,...,...,...
2021-11-26,39.610001,40.619999,38.770000,40.520000,40.520000,35453600
2021-11-29,41.060001,41.250000,38.779999,39.700001,39.700001,27724400
2021-11-30,39.250000,39.950001,37.035000,38.000000,38.000000,55268500


In [6]:
data.to_csv("uber.csv")

In [10]:
import quandl
quandl_api="uY6axD6gnsSCVR3rs61b"
quandl.ApiConfig.api_key = quandl_api

In [13]:
!python -m "pyalgotrade.tools.quandl" --source-code="WIKI" --table-code="ORCL" --from-year=2000 --to-year=2000 --storage=. --force-download --frequency=daily

2021-12-02 21:08:41,757 quandl [INFO] Downloading ORCL 2000 to .\WIKI-ORCL-2000-quandl.csv
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyalgotrade\tools\quandl.py", line 207, in <module>
    main()
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyalgotrade\tools\quandl.py", line 194, in main
    download_daily_bars(args.source_code, args.table_code, year, fileName, args.auth_token)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyalgotrade\tools\quandl.py", line 64, in download_daily_bars
    bars = download_csv(sourceCode, tableCode, datetime.date(year, 1, 1), datetime.date(year, 12, 31), "daily", authToken)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyalgotrade\tools\quandl.py", line 46, in download_csv
 

In [11]:
data = quandl.get("EIA/PET_RWTC_D", collapse="monthly")

In [12]:
data

,Value
Date,
1986-01-31,18.95
1986-02-28,13.23
1986-03-31,10.25
1986-04-30,13.38
1986-05-31,14.30
...,...
2021-07-31,73.93
2021-08-31,68.43
2021-09-30,75.22


In [7]:
from __future__ import print_function

from pyalgotrade import strategy
from pyalgotrade.barfeed import quandlfeed
from pyalgotrade.technical import ma


class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, smaPeriod):
        super(MyStrategy, self).__init__(feed, 1000)
        self.__position = None
        self.__instrument = instrument
        # We'll use adjusted close values instead of regular close values.
        self.setUseAdjustedValues(True)
        self.__sma = ma.SMA(feed[instrument].getPriceDataSeries(), smaPeriod)

    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("BUY at $%.2f" % (execInfo.getPrice()))

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info("SELL at $%.2f" % (execInfo.getPrice()))
        self.__position = None

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        # Wait for enough bars to be available to calculate a SMA.
        if self.__sma[-1] is None:
            return

        bar = bars[self.__instrument]
        # If a position was not opened, check if we should enter a long position.
        if self.__position is None:
            if bar.getPrice() > self.__sma[-1]:
                # Enter a buy market order for 10 shares. The order is good till canceled.
                self.__position = self.enterLong(self.__instrument, 10, True)
        # Check if we have to exit the position.
        elif bar.getPrice() < self.__sma[-1] and not self.__position.exitActive():
            self.__position.exitMarket()


def run_strategy(smaPeriod):
    # Load the bar feed from the CSV file
    feed = quandlfeed.Feed()
    feed.addBarsFromCSV("orcl", "uber.csv")

    # Evaluate the strategy with the feed.
    myStrategy = MyStrategy(feed, "orcl", smaPeriod)
    myStrategy.run()
    print("Final portfolio value: $%.2f" % myStrategy.getBroker().getEquity())

run_strategy(15)

Exception: The barfeed doesn't support adjusted close values